In [2]:
from clickhouse_driver import Client

client = Client(host="192.168.222.220")

In [4]:
factor_database = "factor_database"
factor_table = "factor1"
select_query = "select count(*) from " + factor_database + "." + factor_table
%timeit client.execute(select_query)

114 ms ± 2.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
import DataPlatForm.FactorRead as fread
factor_table = "factor8"
res = fread.read_factor(factor_table, "2006-01-01", "2006-01-06", [])
print(res)

        2006-01-02  2006-01-03  2006-01-04  2006-01-05  2006-01-06
000001    0.697378    0.327902    0.151692    0.059930    0.633121
000002    0.314261    0.949350    0.357480    0.382293    0.702390
000003    0.684868    0.452919    0.548944    0.045304    0.497855
000004    0.741715    0.336021    0.850488    0.100983    0.982476
000005    0.819869    0.198847    0.668904    0.163227    0.087648
...            ...         ...         ...         ...         ...
003996    0.635063    0.085754    0.320411    0.117137    0.634944
003997    0.714408    0.205897    0.073714    0.705919    0.398208
003998    0.140696    0.126257    0.498904    0.581137    0.437986
003999    0.500233    0.772402    0.198684    0.406736    0.222578
004000    0.991414    0.461154    0.509910    0.420753    0.304597

[4000 rows x 5 columns]


In [13]:
client_new = Client(host="192.168.222.221")
database_name = "db"
table_name = "entries"

#show all databases
show_databases_query = "show databases"
res = client_new.execute(show_databases_query)
print(res)

#show all tables in specific databases
use_database_query = "use db"
client_new.execute(use_database_query)
show_table_query = "show tables in db"
res = client_new.execute(use_database_query)
print(res)

[('INFORMATION_SCHEMA',), ('db',), ('default',), ('information_schema',), ('system',)]
[]


In [2]:
#insert 100w row into the cluster
from datetime import timedelta 
from datetime import datetime

#"2005-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")
def gen_insert_string(start_stamp, records_num):
    insert_str = ""
    start_timestamp = datetime.strptime(start_stamp, "%Y-%m-%d %H:%M:%S")
    for i in range(records_num):
        cur_timestamp = start_timestamp + timedelta(seconds=1 * i)
        cur_string = "This is just a test line"
        cur_value = 0.24123
        
        cur_insert_value = "("
        cur_insert_value += "\'" + str(cur_timestamp) + "\'" + ","
        cur_insert_value += "\'" + cur_string + "\'" + ","
        cur_insert_value += str(cur_value)
        cur_insert_value += ")"
        
        insert_str += cur_insert_value + ","
    return insert_str[0:-1]

In [6]:
from clickhouse_driver import Client
import time

client = Client(host="192.168.222.221")
database = "db"
table = "entries6"

start = time.time()
insert_values = gen_insert_string("2005-01-01 00:00:00", 10 * 10000)
print("insert values gen: ", time.time() - start)

insert_query = "insert into db.entries6 Values " + insert_values
start = time.time()
#%timeit 
client.execute(insert_query)
print("insert query cost: ", time.time() - start)

#50w行数据, 3 shards
#insert values gen:  40.53550910949707
#insert cost time:  104.56619119644165

#50w行数据, 2 shards
#insert values gen:  43.18967533111572
#insert cost time:  91.21817660331726

# 1 shards
#insert values gen:  46.318817138671875
#insert cost time:  125.26755285263062

insert values gen:  1.3898341655731201
insert query cost:  65.41122484207153


In [36]:
select_query = "select * from " + database + "." + table + " limit 500000"
start = time.time()
client.execute(select_query)
print("read cost time: ", time.time() - start)
#50w rows read cost
#read cost time(3 shardings):  16.837963581085205

C:\Users\hjiang\AppData\Roaming\Python\Python38\site-packages\clickhouse_driver\columns\datetimecolumn.py:199: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  local_timezone = get_localzone().zone


read cost time:  16.837963581085205
